In [1]:
import sys  
sys.path.insert(1, '.\..\..\SOURCES\.')

import random

random.seed(10)

from DATA import InubushiErgodic

data = InubushiErgodic(nu=-1)
data.delay_timeseries(1)
data.split([80,0,20])

X_TR, Y_TR = data.TR()
X_VAL, Y_VAL = data.VAL()
X_TS, Y_TS = data.TS()


In [7]:
from Configuartions import IPReservoirConfiguration, ReservoirConfiguration
from Evaluator import Evaluator
from Metrics import *
from IPMask import IPMask

N_UNITS_LIST = [50, 100, 150, 200, 250]
repetitions = 10

In [3]:
results = []
evaluator = Evaluator(True, "InubushiTargetDistCompariosn")
for N in N_UNITS_LIST:
    init_point = ReservoirConfiguration(N=N ,desired_rho=0.8, input_scaling=0.5, bias=True, bh_scaling=0.1, bu_scaling=0.1,  name=f"Vanilla") 
    masks = [
        IPMask.gaussian(N),
        IPMask.bimodal(N, mu=0.85, std=0.15),
        IPMask.quadrimodal(N),
        IPMask.trimodal(N, linear_rate=0.6, std_bim=0.1),
        IPMask.trimodal(N, linear_rate=0.8, std_bim=0.15),

        ]
    mask_names = ["Gaussian", "Hard Bimodal", "Soft Bimodal", "Trimodal", "Edgy trimodal" ]
    configs = [init_point] + [IPReservoirConfiguration(init_point, mask=mask, epochs=15, name=f"{name}") for mask,name in zip(masks, mask_names) ]
    results.append(evaluator.evaluate_multiple(configs, data, repetitions, 100, [MSE(), NRMSE()], [MC(), DeltaPhi(), MLLE(X_TS), Neff(), Rho(), KL(X_TS)] ))


results = pd.concat(results)

results.insert(1, "N", np.repeat(N_UNITS_LIST,2*(len(masks)+1)))
results.reset_index(drop=True)
results.to_csv("INUBUSHI-DIST-COMP.csv")

Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps
Reservoir warmed up with the first 100 time steps


In [6]:
results

,Model Name,N,Model Index,Aggregation,MSE,NRMSE,MC,DeltaPhi,MLLE,Neff,Rho,KL
0,Vanilla,5,0.0,Mean,0.000686,1.821712,4.880282,0.058828,-0.299239,1.776349,8.000003e-01,0.000000
1,Vanilla,5,0.0,Std,0.000358,0.513544,0.120334,0.015323,0.033400,0.516513,2.086163e-07,0.000000
2,Gaussian,5,1.0,Mean,0.006145,5.599792,4.432153,0.045994,-0.025994,2.656876,9.096955e-01,0.058703
3,Gaussian,5,1.0,Std,0.001863,0.869482,0.230477,0.001159,0.005416,0.121242,2.682209e-07,0.002490
4,Hard Bimodal,5,2.0,Mean,0.001412,2.456588,2.988424,0.163788,-0.768866,1.357503,9.096953e-01,0.004014
5,Hard Bimodal,5,2.0,Std,0.001089,1.158493,0.177511,0.101526,0.076358,0.001857,2.980232e-07,0.001321
6,Vanilla,10,0.0,Mean,0.000073,0.614276,7.317866,0.044136,-0.258060,2.208447,7.999997e-01,0.000000
7,Vanilla,10,0.0,Std,0.000016,0.068199,0.623282,0.000789,0.002580,0.012385,3.874302e-07,0.000000
8,Gaussian,10,1.0,Mean,0.000772,1.912216,8.000308,0.063940,-0.053148,3.839772,9.096957e-01,0.064629
9,Gaussian,10,1.0,Std,0.000449,0.613813,0.244610,0.007745,0.043770,0.105849,2.980232e-07,0.015367
